# Dates and Times

Being able to handle dates and times, particularly for time series applications, is a critical part of data-cleaning. It is easy to let subtle bugs creep in due to parsing dates and times incorrectly or not accounting for timezones. On rare occasion, you may encounter datasets coming from a system that bravely uses custom time types, like 27-hour clocks (yes, this happened to me!). The point is, working with dates and times can be messy so we will learn some practical strategies here. 



In [1]:
import pandas as pd 
import numpy as np 

We are going to load a dataframe from Github that contains 5 columns of dates and times, all representing the same value but in 5 different formats. 

In [3]:
url = 'https://raw.githubusercontent.com/thomasnield/machine-learning-demo-data/master/timeseries/datetime_formatting.csv'

df = pd.read_csv(url)
df

,RECORD_ID,ORDER_DATE1,ORDER_DATE2,ORDER_DATE_TM1,ORDER_DATE_TM2,ORDER_DATE_TM3
0,1,2022-01-22,1/22/22,1/22/22 16:08,22-Jan-22 4:08 PM,"Sat January 22, 2022 16:08"
1,2,2022-09-25,9/25/22,9/25/22 20:16,22-Sep-25 8:16 PM,"Sun September 25, 2022 20:16"
2,3,2022-07-31,7/31/22,7/31/22 0:32,22-Jul-31 12:32 AM,"Sun July 31, 2022 0:32"
3,4,2022-07-20,7/20/22,7/20/22 11:57,22-Jul-20 11:57 AM,"Wed July 20, 2022 11:57"
4,5,2022-04-26,4/26/22,4/26/22 0:48,22-Apr-26 12:48 AM,"Tue April 26, 2022 0:48"
...,...,...,...,...,...,...
1185,1186,2022-09-27,9/27/22,9/27/22 5:37,22-Sep-27 5:37 AM,"Tue September 27, 2022 5:37"
1186,1187,2022-06-18,6/18/22,6/18/22 1:31,22-Jun-18 1:31 AM,"Sat June 18, 2022 1:31"
1187,1188,2022-08-02,8/2/22,8/2/22 14:54,22-Aug-02 2:54 PM,"Tue August 2, 2022 14:54"
1188,1189,2022-12-11,12/11/22,12/11/22 12:48,22-Dec-11 12:48 PM,"Sun December 11, 2022 12:48"


Let's extract one of the columns and look at its datatypes. Noteice that it is a `dtype` of `object`, not a `datetime64` as we would want. 

In [5]:
df['ORDER_DATE_TM1']

0        1/22/22 16:08
1        9/25/22 20:16
2         7/31/22 0:32
3        7/20/22 11:57
4         4/26/22 0:48
             ...      
1185      9/27/22 5:37
1186      6/18/22 1:31
1187      8/2/22 14:54
1188    12/11/22 12:48
1189    10/18/22 17:51
Name: ORDER_DATE_TM1, Length: 1190, dtype: object

## Implicit Datetime Conversion

If you want to do any useful calendar or time-based logic on these values, you will have to convert them to a different data type. One way to do this is to call Pandas' `to_datetime()` function on that column. Pandas will then do its best to parse the date for that series. 

In [9]:
parsed_col = pd.to_datetime(df['ORDER_DATE_TM1'])
parsed_col

C:\Users\User\AppData\Local\Temp\ipykernel_11756\494649074.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed_col = pd.to_datetime(df['ORDER_DATE_TM1'])


0      2022-01-22 16:08:00
1      2022-09-25 20:16:00
2      2022-07-31 00:32:00
3      2022-07-20 11:57:00
4      2022-04-26 00:48:00
               ...        
1185   2022-09-27 05:37:00
1186   2022-06-18 01:31:00
1187   2022-08-02 14:54:00
1188   2022-12-11 12:48:00
1189   2022-10-18 17:51:00
Name: ORDER_DATE_TM1, Length: 1190, dtype: datetime64[ns]

We can use calendar-based logic to extract properties, like the day of week. 

In [11]:
parsed_col.dt.dayofweek

0       5
1       6
2       6
3       2
4       1
       ..
1185    1
1186    5
1187    1
1188    6
1189    1
Name: ORDER_DATE_TM1, Length: 1190, dtype: int32

If you already know which columns you want to format as dates/times beforehand, you can pass the `parse_dates` parameter to the `read_csv()` function with a list of column names to expect dates/times form. Let's parse all the dates using this approach and analyze the result. For brevity, let's only look at the first three results. 

In [13]:
df_parsed = pd.read_csv(url, 
            parse_dates=['ORDER_DATE1','ORDER_DATE2','ORDER_DATE_TM1','ORDER_DATE_TM2','ORDER_DATE_TM3'])

df_parsed.head(3)

C:\Users\User\AppData\Local\Temp\ipykernel_11756\2571194720.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_parsed = pd.read_csv(url,
C:\Users\User\AppData\Local\Temp\ipykernel_11756\2571194720.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_parsed = pd.read_csv(url,
C:\Users\User\AppData\Local\Temp\ipykernel_11756\2571194720.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_parsed = pd.read_csv(url,


,RECORD_ID,ORDER_DATE1,ORDER_DATE2,ORDER_DATE_TM1,ORDER_DATE_TM2,ORDER_DATE_TM3
0,1,2022-01-22,2022-01-22,2022-01-22 16:08:00,2022-01-22 16:08:00,2022-01-22 16:08:00
1,2,2022-09-25,2022-09-25,2022-09-25 20:16:00,2025-09-22 20:16:00,2022-09-25 20:16:00
2,3,2022-07-31,2022-07-31,2022-07-31 00:32:00,2031-07-22 00:32:00,2022-07-31 00:32:00


Are there any errors? As a matter of fact, the `ORDER_DATE_TM2` was parsed almost completely in error! For example, the second record really has values representing a datetime of `2022-09-25 20:16:00` but the `ORDER_DATE_TM2` was wrongly parsed as `2025-09-22 20:16:00`! What happened? 

Well let's look at the original value. As a matter of fact, let's sample the first three records and analyze what's going on.

In [15]:
df.head(3)

,RECORD_ID,ORDER_DATE1,ORDER_DATE2,ORDER_DATE_TM1,ORDER_DATE_TM2,ORDER_DATE_TM3
0,1,2022-01-22,1/22/22,1/22/22 16:08,22-Jan-22 4:08 PM,"Sat January 22, 2022 16:08"
1,2,2022-09-25,9/25/22,9/25/22 20:16,22-Sep-25 8:16 PM,"Sun September 25, 2022 20:16"
2,3,2022-07-31,7/31/22,7/31/22 0:32,22-Jul-31 12:32 AM,"Sun July 31, 2022 0:32"


What seems to be happening with `ORDER_DATE_TM2` is it is confusing the day and month. Conventionally one might expect a format like `22-Sep-25 8:16 PM` to have the day `22` to be leading, and this is what Pandas assumed. However, a sadistic developer decided to arbitrate their own convention and record the year in that place instead, and put the day `25` after the month. 

This explains why the first record `22-Jan-22 4:08PM` happened to be parsed correctly, as the year and day of month were exactly the same. 

To handle this, we will need to do explicit conversion.

## Explicit Datetime Conversion

Study the datetime formatting conventions for Python here. 

https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior

`strftime()` and `strptime()` are used to write a datetime to a formatting string, and parse a datetime from a formatted string respectively. The format codes come from the standard C conventions. Here are a few common ones, many of which we will use in this notebook. Refer to the link above to see all format codes. 

| Symbol | Description             | Parse Example: 2021-01-13 2:34PM |
|--------|-------------------------|----------------------------------|
| `%y`     | 2-digit year            | 21                               |
| `%m`     | 2-digit month           | 01                               |
| `%d`     | 2-digit day of month    | 13                               |
| `%b`     | 3-letter month          | Jan                              |
| `%I`     | Hour for 12-hour clock  | 2                                |
| `%H`     | Hour for 24-hour clock  | 14                               |
| `%M`     | 2-digit minute          | 34                               |
| `%p`     | AM/PM for 12-hour clock | PM                               |
| `%S`     | 2-digit seconds         | 00                               |
| `%f`     | Microseconds            | 000000                           |
| `%a`     | 3-letter weekday        | Wed                              |
| `%A`     | Full name weekday       | Wednesday                        |

For our problmatic `ORDER_DATE_TM2` column, we need `%y` to get a two-digit year, a `%b` for the three-letter name of the month, and `%d` for the day of month. For the time we use `%I` for the 12-hour clock hour, `%M` for the minute, and `%p` for the `AM/PM`. We will repair the `ORDER_DATE_TM2` parsing by passing it to the `to_datetime()` function along with the `format` parameter, and then assign this back to the parsed dataframe. 

In [19]:
df_parsed['ORDER_DATE_TM2'] = pd.to_datetime(df['ORDER_DATE_TM2'], format='%y-%b-%d %I:%M %p')
df_parsed

,RECORD_ID,ORDER_DATE1,ORDER_DATE2,ORDER_DATE_TM1,ORDER_DATE_TM2,ORDER_DATE_TM3
0,1,2022-01-22,2022-01-22,2022-01-22 16:08:00,2022-01-22 16:08:00,2022-01-22 16:08:00
1,2,2022-09-25,2022-09-25,2022-09-25 20:16:00,2022-09-25 20:16:00,2022-09-25 20:16:00
2,3,2022-07-31,2022-07-31,2022-07-31 00:32:00,2022-07-31 00:32:00,2022-07-31 00:32:00
3,4,2022-07-20,2022-07-20,2022-07-20 11:57:00,2022-07-20 11:57:00,2022-07-20 11:57:00
4,5,2022-04-26,2022-04-26,2022-04-26 00:48:00,2022-04-26 00:48:00,2022-04-26 00:48:00
...,...,...,...,...,...,...
1185,1186,2022-09-27,2022-09-27,2022-09-27 05:37:00,2022-09-27 05:37:00,2022-09-27 05:37:00
1186,1187,2022-06-18,2022-06-18,2022-06-18 01:31:00,2022-06-18 01:31:00,2022-06-18 01:31:00
1187,1188,2022-08-02,2022-08-02,2022-08-02 14:54:00,2022-08-02 14:54:00,2022-08-02 14:54:00
1188,1189,2022-12-11,2022-12-11,2022-12-11 12:48:00,2022-12-11 12:48:00,2022-12-11 12:48:00


Much better! Now all the dates are working. We can verify this by counting the number of rows where there is only one unique value between `ORDER_DATE_TM1`,`ORDER_DATE_TM2`, and `ORDER_DATE_TM3`. Sure enough, all 1190 rows have exactly one unique datetime value in each row. 

In [21]:
np.sum(df_parsed.loc[:,['ORDER_DATE_TM1','ORDER_DATE_TM2','ORDER_DATE_TM3']].nunique(axis=1) == 1)

1190

We can do this same verification for the two plain dates columns to make sure they were parsed correctly too. 

In [23]:
np.sum(df_parsed.loc[:,['ORDER_DATE1','ORDER_DATE2']].nunique(axis=1) == 1)

1190

## Filtering on Datetimes

Let's consolidate our dataframe and rename the columns to just `ORDER_DATE` and `ORDER_DATE_TM`

In [26]:
df = df_parsed[['RECORD_ID','ORDER_DATE1','ORDER_DATE_TM1']] \
    .rename(columns={"ORDER_DATE1": "ORDER_DATE", "ORDER_DATE_TM1": "ORDER_DATE_TM"})

df

,RECORD_ID,ORDER_DATE,ORDER_DATE_TM
0,1,2022-01-22,2022-01-22 16:08:00
1,2,2022-09-25,2022-09-25 20:16:00
2,3,2022-07-31,2022-07-31 00:32:00
3,4,2022-07-20,2022-07-20 11:57:00
4,5,2022-04-26,2022-04-26 00:48:00
...,...,...,...
1185,1186,2022-09-27,2022-09-27 05:37:00
1186,1187,2022-06-18,2022-06-18 01:31:00
1187,1188,2022-08-02,2022-08-02 14:54:00
1188,1189,2022-12-11,2022-12-11 12:48:00


Let's say we wanted to get all records where the day of week is Tuesday. In the `dt.dayofweek` property, `0` is going to be Monday and `7` is going to be Sunday. Numerically, this will make Tuesday be a `1`. We can specify this as a condtion and return all records that fall on Tuesday. 

In [28]:
condition = df['ORDER_DATE'].dt.dayofweek == 1
df[condition]

,RECORD_ID,ORDER_DATE,ORDER_DATE_TM
4,5,2022-04-26,2022-04-26 00:48:00
5,6,2022-11-01,2022-11-01 21:29:00
6,7,2022-12-06,2022-12-06 19:26:00
23,24,2022-11-08,2022-11-08 10:06:00
37,38,2022-04-12,2022-04-12 13:25:00
...,...,...,...
1160,1161,2022-08-09,2022-08-09 20:22:00
1183,1184,2022-10-11,2022-10-11 00:40:00
1185,1186,2022-09-27,2022-09-27 05:37:00
1187,1188,2022-08-02,2022-08-02 14:54:00


You can also quickly filter dates and times using a string format. 

In [30]:
df[df['ORDER_DATE'] >= '2022-06-01']

,RECORD_ID,ORDER_DATE,ORDER_DATE_TM
1,2,2022-09-25,2022-09-25 20:16:00
2,3,2022-07-31,2022-07-31 00:32:00
3,4,2022-07-20,2022-07-20 11:57:00
5,6,2022-11-01,2022-11-01 21:29:00
6,7,2022-12-06,2022-12-06 19:26:00
...,...,...,...
1185,1186,2022-09-27,2022-09-27 05:37:00
1186,1187,2022-06-18,2022-06-18 01:31:00
1187,1188,2022-08-02,2022-08-02 14:54:00
1188,1189,2022-12-11,2022-12-11 12:48:00


In [31]:
df[('2022-06-01 12:35PM' <= df['ORDER_DATE_TM']) & (df['ORDER_DATE_TM'] <= '2022-06-05 8:05PM')]

,RECORD_ID,ORDER_DATE,ORDER_DATE_TM
33,34,2022-06-03,2022-06-03 19:33:00
70,71,2022-06-02,2022-06-02 22:40:00
141,142,2022-06-05,2022-06-05 03:00:00
258,259,2022-06-02,2022-06-02 16:32:00
325,326,2022-06-03,2022-06-03 18:21:00
342,343,2022-06-05,2022-06-05 04:30:00
379,380,2022-06-03,2022-06-03 05:41:00
479,480,2022-06-01,2022-06-01 23:33:00
715,716,2022-06-05,2022-06-05 11:39:00
893,894,2022-06-02,2022-06-02 06:16:00


You can also do more explicit datetime conversions for the start and end bounds. 

In [34]:
start = pd.to_datetime('2022-06-01 12:35PM', format='%Y-%m-%d %I:%M%p')
end = pd.to_datetime('2022-06-05 8:05PM', format='%Y-%m-%d %I:%M%p')

df[(start <= df['ORDER_DATE_TM']) & (df['ORDER_DATE_TM'] <= end)]

,RECORD_ID,ORDER_DATE,ORDER_DATE_TM
33,34,2022-06-03,2022-06-03 19:33:00
70,71,2022-06-02,2022-06-02 22:40:00
141,142,2022-06-05,2022-06-05 03:00:00
258,259,2022-06-02,2022-06-02 16:32:00
325,326,2022-06-03,2022-06-03 18:21:00
342,343,2022-06-05,2022-06-05 04:30:00
379,380,2022-06-03,2022-06-03 05:41:00
479,480,2022-06-01,2022-06-01 23:33:00
715,716,2022-06-05,2022-06-05 11:39:00
893,894,2022-06-02,2022-06-02 06:16:00


## Dealing with Timezones

Likely one of the biggest headaches you can encounter in data cleaning when it comes to dates and times is timezone conversions. It is ideal if datetimes are stored in **coordinated universal times or UTC**, which is an internationally agreed standard for storing datetimes. If times need to be converted locally, then **offsets** are used to express that local time based off the UTC time. This sounds easier than it actually is, because regional laws around the world evolve and change offsets especially due to daylight savings time. 

Thankfully there is a convenient library called `pytz` that Pandas already depends on. It will take care of timezone offsets as well as daylight savings time, even capturing DST laws changed in the past! Let's import it and look at the common timezones. 

In [37]:
import pytz

pytz.common_timezones

['Africa/Abidjan', 'Africa/Accra', 'Africa/Addis_Ababa', 'Africa/Algiers', 'Africa/Asmara', 'Africa/Bamako', 'Africa/Bangui', 'Africa/Banjul', 'Africa/Bissau', 'Africa/Blantyre', 'Africa/Brazzaville', 'Africa/Bujumbura', 'Africa/Cairo', 'Africa/Casablanca', 'Africa/Ceuta', 'Africa/Conakry', 'Africa/Dakar', 'Africa/Dar_es_Salaam', 'Africa/Djibouti', 'Africa/Douala', 'Africa/El_Aaiun', 'Africa/Freetown', 'Africa/Gaborone', 'Africa/Harare', 'Africa/Johannesburg', 'Africa/Juba', 'Africa/Kampala', 'Africa/Khartoum', 'Africa/Kigali', 'Africa/Kinshasa', 'Africa/Lagos', 'Africa/Libreville', 'Africa/Lome', 'Africa/Luanda', 'Africa/Lubumbashi', 'Africa/Lusaka', 'Africa/Malabo', 'Africa/Maputo', 'Africa/Maseru', 'Africa/Mbabane', 'Africa/Mogadishu', 'Africa/Monrovia', 'Africa/Nairobi', 'Africa/Ndjamena', 'Africa/Niamey', 'Africa/Nouakchott', 'Africa/Ouagadougou', 'Africa/Porto-Novo', 'Africa/Sao_Tome', 'Africa/Tripoli', 'Africa/Tunis', 'Africa/Windhoek', 'America/Adak', 'America/Anchorage', 'Amer

Whoa! That's a lot. Let's say we want to localize to Arizona time in the United States. Arizona is kind of special because it does not recognize daylight savings time like the rest of the United States. It gets enough sunlight year round! 

We will look it up by name and save it to a variable. 

In [39]:
tz = pytz.timezone('US/Arizona')
tz

<DstTzInfo 'US/Arizona' LMT-1 day, 16:32:00 STD>

Now let's look at our `ORDER_DATE_TM` column in our dataframe. Notice that there is no information about the timezone, making it **timezone naive**.

In [41]:
if df['ORDER_DATE_TM'].dt.tz is None:
    print("TZ NAIVE")
else:
    df['ORDER_DATE_TM'].dt.tz

TZ NAIVE


Let's say these dates were actually recorded in Arizona local time. We can make `ORDER_DATE_TM` timezone-aware by calling the `tz_localize()` function and specifying they were recorded in `US/Arizona` time. 

In [44]:
df['ORDER_DATE_TM'] = df['ORDER_DATE_TM'].dt.tz_localize('US/Arizona')
df['ORDER_DATE_TM']

0      2022-01-22 16:08:00-07:00
1      2022-09-25 20:16:00-07:00
2      2022-07-31 00:32:00-07:00
3      2022-07-20 11:57:00-07:00
4      2022-04-26 00:48:00-07:00
                  ...           
1185   2022-09-27 05:37:00-07:00
1186   2022-06-18 01:31:00-07:00
1187   2022-08-02 14:54:00-07:00
1188   2022-12-11 12:48:00-07:00
1189   2022-10-18 17:51:00-07:00
Name: ORDER_DATE_TM, Length: 1190, dtype: datetime64[ns, US/Arizona]

Notice how the datatype of `ORDER_DATE_TM` changes to `datetime64[ns, US/Arizona]`, making it no longer timezone naive and now associated with Arizona time. The `-07:00` part of the value indicates the offset from UTC. 

We are now free to convert it to different timezones. Let's say we wanted to add an additional column `ORDER_DATE_TM_CST` showing the datetime in US/Central time. We can use `tz_convert` to perform this conversion. Notice below how central time is sometimes 2 hours ahead of Arizona, but also sometimes 1 hour ahead. This is because Arizona does not observe daylight savings time but central time does. 

In [46]:
df['ORDER_DATE_TM_CST'] = df['ORDER_DATE_TM'].dt.tz_convert('US/Central')
df

,RECORD_ID,ORDER_DATE,ORDER_DATE_TM,ORDER_DATE_TM_CST
0,1,2022-01-22,2022-01-22 16:08:00-07:00,2022-01-22 17:08:00-06:00
1,2,2022-09-25,2022-09-25 20:16:00-07:00,2022-09-25 22:16:00-05:00
2,3,2022-07-31,2022-07-31 00:32:00-07:00,2022-07-31 02:32:00-05:00
3,4,2022-07-20,2022-07-20 11:57:00-07:00,2022-07-20 13:57:00-05:00
4,5,2022-04-26,2022-04-26 00:48:00-07:00,2022-04-26 02:48:00-05:00
...,...,...,...,...
1185,1186,2022-09-27,2022-09-27 05:37:00-07:00,2022-09-27 07:37:00-05:00
1186,1187,2022-06-18,2022-06-18 01:31:00-07:00,2022-06-18 03:31:00-05:00
1187,1188,2022-08-02,2022-08-02 14:54:00-07:00,2022-08-02 16:54:00-05:00
1188,1189,2022-12-11,2022-12-11 12:48:00-07:00,2022-12-11 13:48:00-06:00


Finally, we can of course convert the date to `UTC` which we will save to a column `ORDER_DATE_TM_UTC`. Notice the `+00:00` offset as UTC is the baseline with no offset.

In [48]:
df['ORDER_DATE_TM_UTC'] = df['ORDER_DATE_TM'].dt.tz_convert('UTC')
df

,RECORD_ID,ORDER_DATE,ORDER_DATE_TM,ORDER_DATE_TM_CST,ORDER_DATE_TM_UTC
0,1,2022-01-22,2022-01-22 16:08:00-07:00,2022-01-22 17:08:00-06:00,2022-01-22 23:08:00+00:00
1,2,2022-09-25,2022-09-25 20:16:00-07:00,2022-09-25 22:16:00-05:00,2022-09-26 03:16:00+00:00
2,3,2022-07-31,2022-07-31 00:32:00-07:00,2022-07-31 02:32:00-05:00,2022-07-31 07:32:00+00:00
3,4,2022-07-20,2022-07-20 11:57:00-07:00,2022-07-20 13:57:00-05:00,2022-07-20 18:57:00+00:00
4,5,2022-04-26,2022-04-26 00:48:00-07:00,2022-04-26 02:48:00-05:00,2022-04-26 07:48:00+00:00
...,...,...,...,...,...
1185,1186,2022-09-27,2022-09-27 05:37:00-07:00,2022-09-27 07:37:00-05:00,2022-09-27 12:37:00+00:00
1186,1187,2022-06-18,2022-06-18 01:31:00-07:00,2022-06-18 03:31:00-05:00,2022-06-18 08:31:00+00:00
1187,1188,2022-08-02,2022-08-02 14:54:00-07:00,2022-08-02 16:54:00-05:00,2022-08-02 21:54:00+00:00
1188,1189,2022-12-11,2022-12-11 12:48:00-07:00,2022-12-11 13:48:00-06:00,2022-12-11 19:48:00+00:00


Isn't it nice how much work the `tz` library did for you? It is encouraged to use it as it will maintain that complex database of timezones, offsets, daylight savings, and historical changes to daylight savings laws for you. 

## EXERCISE

A dataframe of two columns and two records is shown below. Complete the code below by replacing the question marks `?` so that `ORDER_DATE_TM` is localized to `US/Pacific`. Then add a new column `ORDER_DATE_TM_PARIS` that shows the equivalent time in `Europe/Paris`.  

In [55]:
import pandas as pd

df = pd.DataFrame({
    "ORDER_ID": [1, 2], 
    "ORDER_DATE_TM": [pd.to_datetime('2023-01-05 7:05 PM'), pd.to_datetime('2023-01-06 8:15 AM')]
})

# localize to US/Pacific
df["ORDER_DATE_TM"] = df["ORDER_DATE_TM"].dt.tz_localize('US/Pacific')

# convert to Europe/Paris 
df["ORDER_DATE_TM_PARIS"] = df["ORDER_DATE_TM"].dt.tz_convert('Europe/Paris')

df

,ORDER_ID,ORDER_DATE_TM,ORDER_DATE_TM_PARIS
0,1,2023-01-05 19:05:00-08:00,2023-01-06 04:05:00+01:00
1,2,2023-01-06 08:15:00-08:00,2023-01-06 17:15:00+01:00
